# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |  Mateo Tavares Trueba |
| **Fecha**      | 10 de Septiembre 2025  |
| **Expediente** |  746004 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [27]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from scipy import stats 

In [32]:
adv=pd.read_csv('advertising.csv')

In [33]:
adv.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [34]:
adv = adv.drop(columns=["Unnamed: 0"])

Penalización L2.

In [35]:
X = adv[["TV", "radio", "newspaper"]]
y = adv["sales"]

In [38]:
reg1 = Ridge(alpha=1.0) 
reg1.fit(X, y)

Ridge()

In [39]:
y_pred = reg.predict(X)
r21 = r2_score(y, y_pred)
print("R^2:", r21)
print("Intercepto:", reg.intercept_)
print("Coeficientes (TV, radio, newspaper):", reg.coef_)

R^2: 0.8972106380074802
Intercepto: 2.9389674583301524
Coeficientes (TV, radio, newspaper): [ 0.04576464  0.1885251  -0.00103629]


In [41]:
y_mean = np.mean(y)
TSS1 = np.sum((y - y_mean) ** 2)
RSS1 = (1-r21) * TSS1
print('RSS:',RSS1)
RSE1= np.sqrt(RSS1/(len(y)-len(reg3.coef_)))
p=4
RSE1= RSS1/((len(y)-p))
n=len(y)
unos=np.ones([n,1])
X = np.hstack([
    unos,
    adv['TV'].values.reshape(-1, 1),
    adv['radio'].values.reshape(-1, 1),
    adv['newspaper'].values.reshape(-1, 1)
])
var_beta1 = np.linalg.inv(X.T @ X) *RSE**2
var_beta1
std_beta1 = np.sqrt(var_beta1.diagonal())
std_beta1
t1_b0 = reg1.intercept_/std_beta1[0]
t1_b1 = reg1.coef_[0]/std_beta1[1]
t1_b2 = reg1.coef_[1]/std_beta1[2]
t1_b3 = reg1.coef_[2]/std_beta1[3]
t1_b0, t1_b1, t1_b2, t1_b3

RSS: 556.825263831076


(5.5903178925342285,
 19.465097134147122,
 12.988899412437426,
 -0.10472219648901107)

In [42]:
p_b01= 2*(1-stats.t.cdf(np.abs(t_b0), df=n-2))
p_b11= 2*(1-stats.t.cdf(np.abs(t_b1), df=n-2))
p_b21= 2*(1-stats.t.cdf(np.abs(t_b2), df=n-2))
p_b31= 2*(1-stats.t.cdf(np.abs(t_b3), df=n-2))
p_b01, p_b11, p_b21, p_b31

(7.445264627037318e-08, 0.0, 0.0, 0.9167022547014259)

Con Penalización L1.

In [16]:
reg2 = Lasso(alpha=0.1) 
reg2.fit(X, y)
y_pred = reg2.predict(X)
r22 = r2_score(y, y_pred)
print("R^2 :", r22)
print("Intercepto:", reg2.intercept_)
print("Coeficientes (TV, radio, newspaper):", reg2.coef_)

R^2 : 0.8972068586756202
Intercepto: 2.9444386596073855
Coeficientes (TV, radio, newspaper): [ 0.04575172  0.18788735 -0.00066758]


In [43]:
y_mean = np.mean(y)
TSS2 = np.sum((y - y_mean) ** 2)
RSS2 = (1-r22) * TSS2
print('RSS:',RSS2)
RSE2= np.sqrt(RSS2/(len(y)-len(reg2.coef_)))
p=4
RSE2= RSS2/((len(y)-p))
n=len(y)
unos=np.ones([n,1])
X = np.hstack([
    unos,
    adv['TV'].values.reshape(-1, 1),
    adv['radio'].values.reshape(-1, 1),
    adv['newspaper'].values.reshape(-1, 1)
])
var_beta2 = np.linalg.inv(X.T @ X) *RSE2**2
var_beta2
std_beta2 = np.sqrt(var_beta2.diagonal())
std_beta2
t2_b0 = reg2.intercept_/std_beta2[0]
t2_b1 = reg2.coef_[0]/std_beta2[1]
t2_b2 = reg2.coef_[1]/std_beta2[2]
t2_b3 = reg2.coef_[2]/std_beta2[3]
t2_b0, t2_b1, t2_b2, t2_b3

RSS: 556.8457370339373


(5.600518937602108, 19.458884351459815, 12.94448430160039, -0.067459011163849)

In [44]:
p_b02= 2*(1-stats.t.cdf(np.abs(t2_b0), df=n-2))
p_b12= 2*(1-stats.t.cdf(np.abs(t2_b1), df=n-2))
p_b22= 2*(1-stats.t.cdf(np.abs(t2_b2), df=n-2))
p_b32= 2*(1-stats.t.cdf(np.abs(t2_b3), df=n-2))
p_b02, p_b12, p_b22, p_b32

(7.076778008041629e-08, 0.0, 0.0, 0.9462843629468995)

Sin penalización.

In [18]:
reg3 = LinearRegression()
reg3.fit(X, y)
y_pred = reg3.predict(X)
r23 = r2_score(y, y_pred)
print("R^2:", r23)
print("Intercepto:", reg3.intercept_)
print("Coeficientes (TV, radio, newspaper):", reg3.coef_)


R^2: 0.8972106381789522
Intercepto: 2.9388893694594067
Coeficientes (TV, radio, newspaper): [ 0.04576465  0.18853002 -0.00103749]


In [45]:
y_mean = np.mean(y)
TSS3 = np.sum((y - y_mean) ** 2)
RSS3 = (1-r23) * TSS3
print('RSS:',RSS3)
RSE= np.sqrt(RSS3/(len(y)-len(reg3.coef_)))
p=4
RSE= RSS3/((len(y)-p))
n=len(y)
unos=np.ones([n,1])
X = np.hstack([
    unos,
    adv['TV'].values.reshape(-1, 1),
    adv['radio'].values.reshape(-1, 1),
    adv['newspaper'].values.reshape(-1, 1)
])
var_beta3 = np.linalg.inv(X.T @ X) *RSE**2
var_beta3
std_beta3 = np.sqrt(var_beta3.diagonal())
std_beta3
t3_b0 = reg3.intercept_/std_beta3[0]
t3_b1 = reg3.coef_[0]/std_beta3[1]
t3_b2 = reg3.coef_[1]/std_beta3[2]
t3_b3 = reg3.coef_[2]/std_beta3[3]
t3_b0, t3_b1, t3_b2, t3_b3

RSS: 556.8252629021869


(5.590169356826526,
 19.465097898627228,
 12.989238395317944,
 -0.10484336931268624)

In [46]:
p3_b03= 2*(1-stats.t.cdf(np.abs(t3_b0), df=n-2))
p3_b13= 2*(1-stats.t.cdf(np.abs(t3_b1), df=n-2))
p3_b23= 2*(1-stats.t.cdf(np.abs(t3_b2), df=n-2))
p3_b33= 2*(1-stats.t.cdf(np.abs(t3_b3), df=n-2))
p3_b03, p3_b13, p3_b23, p3_b33

(7.450766026373401e-08, 0.0, 0.0, 0.9166062260539096)

Comparamos los r2 scores:

In [29]:
print('R2 Score L2:',r21)
print('R2 Score L1:',r22)
print('R2 Score sin penalización:',r23)

R2 Score L2: 0.8972106380074802
R2 Score L1: 0.8972068586756202
R2 Score sin penalización: 0.8972106381789522


En cuanto a r2, todas se mantienen casi idénticas.

Comparamos los P values.

In [47]:
print('P_values L2 b0,b1,b2,b3:',p_b01, p_b11, p_b21, p_b31)
print('P_values L1 b0,b1,b2,b3:',p_b02, p_b12, p_b22, p_b32)
print('P_values sin penalización b0,b1,b2,b3:',p3_b03, p3_b13, p3_b23, p3_b33)

P_values L2 b0,b1,b2,b3: 7.445264627037318e-08 0.0 0.0 0.9167022547014259
P_values L1 b0,b1,b2,b3: 7.076778008041629e-08 0.0 0.0 0.9462843629468995
P_values sin penalización b0,b1,b2,b3: 7.450766026373401e-08 0.0 0.0 0.9166062260539096


Podemos ver entonces que el intercepto es casi irrelevante dependiendo de la significancia en todas las regresiones, nos arroja un p_Value del 7%, así también el newspaper con un p_value de 91%. Por tanto concluimos que las 2 variables que explican las ventas son TV (b1) y radio(b2)